In [1]:
import pandas as pd
from PIL import Image,ImageDraw,ImageFont
import os
import numpy as np
import math

#### train matrix

In [2]:
train_matrix = pd.read_csv("170728_2015-01-01_train-matrix-and-labels.csv")
labels = train_matrix.outcome
del train_matrix['outcome']

## Binning

In [3]:
def binticks_label(matrix,feat):
    tmp = matrix.loc[:,feat]
    tmpmin = min(tmp)
    tmpmax = max(tmp)
    tmprange = tmpmax - tmpmin
    step = tmprange / 100
    return np.arange(tmpmin,tmpmax+step,step)

In [4]:
def labelticks(matrix,feat):
    tmp = matrix.loc[:,feat]
    tmpmin = min(tmp)
    tmpmax = max(tmp)
    tmprange = tmpmax - tmpmin
    step = tmprange / 20
    return np.arange(tmpmin,tmpmax+step,step)

In [5]:
def minmax_label(matrix,feat):
    tmp = matrix.loc[:,feat]
    tmpmin = min(tmp)
    tmpmax = max(tmp)
    return tmpmin,tmpmax

### Axis labels

In [6]:
def addlabels(canvas,ticks):
    width = canvas.width
    height = canvas.height
    margin = 28
    newheight = height + margin
    
    unicanvas = Image.new('RGB',(width,newheight),(255,255,255))
    unicanvas.paste(canvas,(0,0))
    
    font = ImageFont.truetype('VeraMono.ttf', rectheight - 8 )
    
    # fontWidth, fontHeight = font.getsize(high) # can use if I have sizing issues

    draw = ImageDraw.Draw(unicanvas,'RGB')
    
    for i in range(len(ticks)):
        xpos = i * rectwidth * 5
        draw.text((xpos,newheight - 28 ),text=str(ticks[i]),font=font,fill=(0,0,0))
    
    return unicanvas

# Train plot

### Overlaid histograms

In [11]:
cfeat = "dispatch_id_p3m_dispatchtype_traffic_sum"
ifeat = "demarrests_id_p2y_blackpopulation_avg"

In [12]:
binned = pd.cut(train_matrix[ifeat],bins=4,labels=False)

In [13]:
subset0 = train_matrix[binned==0]
subset1 = train_matrix[binned==1]
subset2 = train_matrix[binned==2]
subset3 = train_matrix[binned==3]

In [20]:
def overhist_label(X,labels,feat,rectwidth,rectheight,pad,subset):
    
    pos = pd.cut(subset[labels==1].loc[:,feat],bins=binticks_label(X,feat),labels=False,include_lowest=True)
    neg = pd.cut(subset[labels==0].loc[:,feat],bins=binticks_label(X,feat),labels=False,include_lowest=True)
    
    # handmade binticks
    #pos = pd.cut(subset[labels==1].loc[:,feat],bins=np.arange(22,69,1),labels=False,include_lowest=True)
    #neg = pd.cut(subset[labels==0].loc[:,feat],bins=np.arange(22,69,1),labels=False,include_lowest=True)
    
    binmax = neg.value_counts().max()
    
    px_w = len(binticks_label(X,feat)) * rectwidth + 1
    px_h = binmax * rectheight
    
    canvas = Image.new('RGB',(px_w,px_h),(255,255,255))
    draw = ImageDraw.Draw(canvas,'RGBA')
    
    for binn in pos.value_counts().keys():
        xpos = binn * rectwidth
        ypos = px_h - rectheight

        tmp = pos[pos==binn]

        for i in tmp.index:
            bbox = [(xpos+pad,ypos+pad-1),(xpos+rectwidth-pad,ypos+rectheight-pad-1)]
            draw.rectangle(bbox,fill=(220,101,113,127),outline=None)
            ypos = ypos - rectheight
    
    for binn in neg.value_counts().keys():
        xpos = binn * rectwidth
        ypos = px_h - rectheight

        tmp = neg[neg==binn]
        negbinheight = len(tmp)
        nneg = len(neg)
        npos = len(pos)
        overheight = int( negbinheight * npos / nneg )

        for i in tmp.index:
            bbox = [(xpos,ypos-1),(xpos+rectwidth,ypos+rectheight-1)]
            draw.rectangle(bbox,fill=None,outline=(189,189,189,255))
            ypos = ypos - rectheight
        
        ypos = px_h - rectheight # reset
        
        for i in range(overheight):
            bbox = [(xpos+pad,ypos+pad-1),(xpos+rectwidth-pad,ypos+rectheight-pad-1)]
            draw.rectangle(bbox,fill=(112,159,210,127),outline=None)
            ypos = ypos - rectheight
    
    # range labels
    canvas = addlabels(canvas,labelticks(X,feat))
    
    return canvas

In [21]:
pad = 1

In [22]:
rectwidth = 28
rectheight = 28

In [28]:
subset = subset3

In [29]:
im = overhist_label(train_matrix,labels,cfeat,rectwidth,rectheight,pad,subset)
im.save("/Users/damoncrockett/Desktop/subset3.png")

/Library/Python/2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
